# Machine Learning in Julia

An introduction to the
[MLJ](https://alan-turing-institute.github.io/MLJ.jl/stable/)
toolbox.

### Set-up

Inspect Julia version:

In [1]:
VERSION

v"1.7.1"

The following instantiates a package environment.

The package environment has been created using **Julia 1.6** and may not
instantiate properly for other Julia versions.

In [2]:
using Pkg
Pkg.activate("env")
Pkg.instantiate()

  Activating project at `~/GoogleDrive/Julia/MLJ/MLJTutorial/notebooks/01_data_representation/env`
┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ ~/GoogleDrive/Julia/MLJ/MLJTutorial/notebooks/01_data_representation/env/Manifest.toml:0


## General resources

- [MLJ Cheatsheet](https://alan-turing-institute.github.io/MLJ.jl/dev/mlj_cheatsheet/)
- [Common MLJ Workflows](https://alan-turing-institute.github.io/MLJ.jl/dev/common_mlj_workflows/)
- [MLJ manual](https://alan-turing-institute.github.io/MLJ.jl/dev/)
- [Data Science Tutorials in Julia](https://juliaai.github.io/DataScienceTutorials.jl/)

## Part 1 - Data Representation

> **Goals:**
> 1. Learn how MLJ specifies it's data requirements using "scientific" types
> 2. Understand the options for representing tabular data
> 3. Learn how to inspect and fix the representation of data to meet MLJ requirements

### Scientific types

To help you focus on the intended *purpose* or *interpretation* of
data, MLJ models specify data requirements using *scientific types*,
instead of machine types. An example of a scientific type is
`OrderedFactor`. The other basic "scalar" scientific types are
illustrated below:

![](scitypes.png)

A scientific type is an ordinary Julia type (so it can be used for
method dispatch, for example) but it usually has no instances. The
`scitype` function is used to articulate MLJ's convention about how
different machine types will be interpreted by MLJ models:

In [3]:
using ScientificTypes
scitype(3.141)

Continuous

In [4]:
time = [2.3, 4.5, 4.2, 1.8, 7.1]
scitype(time)

AbstractVector{Continuous} (alias for AbstractArray{Continuous, 1})

To fix data which MLJ is interpreting incorrectly, we use the
`coerce` method:

In [5]:
height = [185, 153, 163, 114, 180]
scitype(height)

AbstractVector{Count} (alias for AbstractArray{Count, 1})

In [6]:
height = coerce(height, Continuous)

5-element Vector{Float64}:
 185.0
 153.0
 163.0
 114.0
 180.0

Here's an example of data we would want interpreted as
`OrderedFactor` but isn't:

In [7]:
exam_mark = ["rotten", "great", "bla",  missing, "great"]
scitype(exam_mark)

AbstractVector{Union{Missing, Textual}} (alias for AbstractArray{Union{Missing, Textual}, 1})

In [8]:
exam_mark = coerce(exam_mark, OrderedFactor)

┌ Info: Trying to coerce from `Union{Missing, String}` to `OrderedFactor`.
└ Coerced to `Union{Missing,OrderedFactor}` instead.


5-element CategoricalArrays.CategoricalArray{Union{Missing, String},1,UInt32}:
 "rotten"
 "great"
 "bla"
 missing
 "great"

In [9]:
levels(exam_mark)

3-element Vector{String}:
 "bla"
 "great"
 "rotten"

Use `levels!` to put the classes in the right order:

In [10]:
levels!(exam_mark, ["rotten", "bla", "great"])
exam_mark[1] < exam_mark[2]

true

When sub-sampling, no levels are lost:

In [11]:
levels(exam_mark[1:2])

3-element Vector{String}:
 "rotten"
 "bla"
 "great"

**Note on binary data.** There is no separate scientific type for
binary data. Binary data is `OrderedFactor{2}` or
`Multiclass{2}`. If a binary measure like `truepositive` is a
applied to `OrderedFactor{2}` then the "positive" class is assumed
to appear *second* in the ordering. If such a measure is applied to
`Multiclass{2}` data, a warning is issued. A single `OrderedFactor`
can be coerced to a single `Continuous` variable, for models that
require this, while a `Multiclass` variable can only be one-hot
encoded.

### Two-dimensional data

Whenever it makes sense, MLJ Models generally expect two-dimensional
data to be *tabular*. All the tabular formats implementing the
[Tables.jl API](https://juliadata.github.io/Tables.jl/stable/) (see
this
[list](https://github.com/JuliaData/Tables.jl/blob/master/INTEGRATIONS.md))
have a scientific type of `Table` and can be used with such models.

Probably the simplest example of a table is the julia native *column
table*, which is just a named tuple of equal-length vectors:

In [12]:
column_table = (h=height, e=exam_mark, t=time)

(h = [185.0, 153.0, 163.0, 114.0, 180.0],
 e = Union{Missing, CategoricalArrays.CategoricalValue{String, UInt32}}["rotten", "great", "bla", missing, "great"],
 t = [2.3, 4.5, 4.2, 1.8, 7.1],)

In [13]:
scitype(column_table)

Table{Union{AbstractVector{Union{Missing, OrderedFactor{3}}}, AbstractVector{Continuous}}}

Notice the `Table{K}` type parameter `K` encodes the scientific
types of the columns. (This is useful when comparing table scitypes
with `<:`). To inspect the individual column scitypes, we use the
`schema` method instead:

In [14]:
schema(column_table)

┌───────┬──────────────────────────────────┬────────────────────────────────────────────────
│ names │ scitypes                         │ types                                         ⋯
├───────┼──────────────────────────────────┼────────────────────────────────────────────────
│ h     │ Continuous                       │ Float64                                       ⋯
│ e     │ Union{Missing, OrderedFactor{3}} │ Union{Missing, CategoricalValue{String, UInt3 ⋯
│ t     │ Continuous                       │ Float64                                       ⋯
└───────┴──────────────────────────────────┴────────────────────────────────────────────────
                                                                            1 column omitted


Here are five other examples of tables:

In [15]:
dict_table = Dict(:h => height, :e => exam_mark, :t => time)
schema(dict_table)

┌───────┬──────────────────────────────────┬────────────────────────────────────────────────
│ names │ scitypes                         │ types                                         ⋯
├───────┼──────────────────────────────────┼────────────────────────────────────────────────
│ e     │ Union{Missing, OrderedFactor{3}} │ Union{Missing, CategoricalValue{String, UInt3 ⋯
│ h     │ Continuous                       │ Float64                                       ⋯
│ t     │ Continuous                       │ Float64                                       ⋯
└───────┴──────────────────────────────────┴────────────────────────────────────────────────
                                                                            1 column omitted


(To control column order here, instead use `LittleDict` from
OrderedCollections.jl.)

In [16]:
row_table = [(a=1, b=3.4),
             (a=2, b=4.5),
             (a=3, b=5.6)]
schema(row_table)

┌───────┬────────────┬─────────┐
│ names │ scitypes   │ types   │
├───────┼────────────┼─────────┤
│ a     │ Count      │ Int64   │
│ b     │ Continuous │ Float64 │
└───────┴────────────┴─────────┘


In [17]:
import DataFrames
df = DataFrames.DataFrame(column_table)

,h,e,t
,Float64,Cat…?,Float64
1,185.0,rotten,2.3
2,153.0,great,4.5
3,163.0,bla,4.2
4,114.0,missing,1.8
5,180.0,great,7.1


In [18]:
schema(df) == schema(column_table)

true

In [19]:
using UrlDownload, CSV
csv_file = urldownload("https://raw.githubusercontent.com/ablaom/"*
                   "MachineLearningInJulia2020/"*
                   "for-MLJ-version-0.16/data/horse.csv");
schema(csv_file)

┌─────────────────────────┬────────────┬─────────┐
│ names                   │ scitypes   │ types   │
├─────────────────────────┼────────────┼─────────┤
│ surgery                 │ Count      │ Int64   │
│ age                     │ Count      │ Int64   │
│ rectal_temperature      │ Continuous │ Float64 │
│ pulse                   │ Count      │ Int64   │
│ respiratory_rate        │ Count      │ Int64   │
│ temperature_extremities │ Count      │ Int64   │
│ mucous_membranes        │ Count      │ Int64   │
│ capillary_refill_time   │ Count      │ Int64   │
│ pain                    │ Count      │ Int64   │
│ peristalsis             │ Count      │ Int64   │
│ abdominal_distension    │ Count      │ Int64   │
│ packed_cell_volume      │ Continuous │ Float64 │
│ total_protein           │ Continuous │ Float64 │
│ outcome                 │ Count      │ Int64   │
│ surgical_lesion         │ Count      │ Int64   │
│ cp_data                 │ Count      │ Int64   │
└─────────────────────────┴────

Most MLJ models do not accept matrix in lieu of a table, but you can
wrap a matrix as a table:

In [20]:
using Tables
matrix_table = Tables.table(rand(2,3))
schema(matrix_table)

┌─────────┬────────────┬─────────┐
│ names   │ scitypes   │ types   │
├─────────┼────────────┼─────────┤
│ Column1 │ Continuous │ Float64 │
│ Column2 │ Continuous │ Float64 │
│ Column3 │ Continuous │ Float64 │
└─────────┴────────────┴─────────┘


The matrix is *not* copied, only wrapped. Some models may perform
better if one wraps the adjoint of the transpose - see
[here](https://alan-turing-institute.github.io/MLJ.jl/dev/getting_started/#Observations-correspond-to-rows,-not-columns).

**Manipulating tabular data.** In this workshop we assume
familiarity with some kind of tabular data container (although it is
possible, in principle, to carry out the exercises without this.)
For a quick start introduction to `DataFrames`, see [this
tutorial](https://juliaai.github.io/DataScienceTutorials.jl/data/dataframe/).

### Fixing scientific types in tabular data

To show how we can correct the scientific types of data in tables,
we introduce a cleaned up version of the UCI Horse Colic Data Set
(the cleaning work-flow is described
[here](https://juliaai.github.io/DataScienceTutorials.jl/end-to-end/horse/#dealing_with_missing_values)).
We already downloaded this data set immediately above.q

In [21]:
horse = DataFrames.DataFrame(csv_file); # convert to data frame
first(horse, 4)

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_extremities,mucous_membranes
,Int64,Int64,Float64,Int64,Int64,Int64,Int64
1,2,1,38.5,66,66,3,1
2,1,1,39.2,88,88,3,4
3,2,1,38.3,40,40,1,3
4,1,9,39.1,164,164,4,6


From [the UCI
docs](http://archive.ics.uci.edu/ml/datasets/Horse+Colic) we can
surmise how each variable ought to be interpreted (a step in our
work-flow that cannot reliably be left to the computer):

variable                    | scientific type (interpretation)
----------------------------|-----------------------------------
`:surgery`                  | Multiclass
`:age`                      | Multiclass
`:rectal_temperature`       | Continuous
`:pulse`                    | Continuous
`:respiratory_rate`         | Continuous
`:temperature_extremities`  | OrderedFactor
`:mucous_membranes`         | Multiclass
`:capillary_refill_time`    | Multiclass
`:pain`                     | OrderedFactor
`:peristalsis`              | OrderedFactor
`:abdominal_distension`     | OrderedFactor
`:packed_cell_volume`       | Continuous
`:total_protein`            | Continuous
`:outcome`                  | Multiclass
`:surgical_lesion`          | OrderedFactor
`:cp_data`                  | Multiclass

Let's see how MLJ will actually interpret the data, as it is
currently encoded:

In [22]:
schema(horse)

┌─────────────────────────┬────────────┬─────────┐
│ names                   │ scitypes   │ types   │
├─────────────────────────┼────────────┼─────────┤
│ surgery                 │ Count      │ Int64   │
│ age                     │ Count      │ Int64   │
│ rectal_temperature      │ Continuous │ Float64 │
│ pulse                   │ Count      │ Int64   │
│ respiratory_rate        │ Count      │ Int64   │
│ temperature_extremities │ Count      │ Int64   │
│ mucous_membranes        │ Count      │ Int64   │
│ capillary_refill_time   │ Count      │ Int64   │
│ pain                    │ Count      │ Int64   │
│ peristalsis             │ Count      │ Int64   │
│ abdominal_distension    │ Count      │ Int64   │
│ packed_cell_volume      │ Continuous │ Float64 │
│ total_protein           │ Continuous │ Float64 │
│ outcome                 │ Count      │ Int64   │
│ surgical_lesion         │ Count      │ Int64   │
│ cp_data                 │ Count      │ Int64   │
└─────────────────────────┴────

As a first correction step, we can get MLJ to "guess" the
appropriate fix, using the `autotype` method:

In [23]:
autotype(horse)

Dict{Symbol, Type} with 11 entries:
  :abdominal_distension    => OrderedFactor
  :pain                    => OrderedFactor
  :surgery                 => OrderedFactor
  :mucous_membranes        => OrderedFactor
  :surgical_lesion         => OrderedFactor
  :outcome                 => OrderedFactor
  :capillary_refill_time   => OrderedFactor
  :age                     => OrderedFactor
  :temperature_extremities => OrderedFactor
  :peristalsis             => OrderedFactor
  :cp_data                 => OrderedFactor

Okay, this is not perfect, but a step in the right direction, which
we implement like this:

In [24]:
coerce!(horse, autotype(horse));
schema(horse)

┌─────────────────────────┬──────────────────┬─────────────────────────────────┐
│ names                   │ scitypes         │ types                           │
├─────────────────────────┼──────────────────┼─────────────────────────────────┤
│ surgery                 │ OrderedFactor{2} │ CategoricalValue{Int64, UInt32} │
│ age                     │ OrderedFactor{2} │ CategoricalValue{Int64, UInt32} │
│ rectal_temperature      │ Continuous       │ Float64                         │
│ pulse                   │ Count            │ Int64                           │
│ respiratory_rate        │ Count            │ Int64                           │
│ temperature_extremities │ OrderedFactor{4} │ CategoricalValue{Int64, UInt32} │
│ mucous_membranes        │ OrderedFactor{6} │ CategoricalValue{Int64, UInt32} │
│ capillary_refill_time   │ OrderedFactor{3} │ CategoricalValue{Int64, UInt32} │
│ pain                    │ OrderedFactor{5} │ CategoricalValue{Int64, UInt32} │
│ peristalsis             │ 

All remaining `Count` data should be `Continuous`:

In [25]:
coerce!(horse, Count => Continuous);
schema(horse)

┌─────────────────────────┬──────────────────┬─────────────────────────────────┐
│ names                   │ scitypes         │ types                           │
├─────────────────────────┼──────────────────┼─────────────────────────────────┤
│ surgery                 │ OrderedFactor{2} │ CategoricalValue{Int64, UInt32} │
│ age                     │ OrderedFactor{2} │ CategoricalValue{Int64, UInt32} │
│ rectal_temperature      │ Continuous       │ Float64                         │
│ pulse                   │ Continuous       │ Float64                         │
│ respiratory_rate        │ Continuous       │ Float64                         │
│ temperature_extremities │ OrderedFactor{4} │ CategoricalValue{Int64, UInt32} │
│ mucous_membranes        │ OrderedFactor{6} │ CategoricalValue{Int64, UInt32} │
│ capillary_refill_time   │ OrderedFactor{3} │ CategoricalValue{Int64, UInt32} │
│ pain                    │ OrderedFactor{5} │ CategoricalValue{Int64, UInt32} │
│ peristalsis             │ 

We'll correct the remaining truant entries manually:

In [26]:
coerce!(horse,
        :surgery               => Multiclass,
        :age                   => Multiclass,
        :mucous_membranes      => Multiclass,
        :capillary_refill_time => Multiclass,
        :outcome               => Multiclass,
        :cp_data               => Multiclass);
schema(horse)

┌─────────────────────────┬──────────────────┬─────────────────────────────────┐
│ names                   │ scitypes         │ types                           │
├─────────────────────────┼──────────────────┼─────────────────────────────────┤
│ surgery                 │ Multiclass{2}    │ CategoricalValue{Int64, UInt32} │
│ age                     │ Multiclass{2}    │ CategoricalValue{Int64, UInt32} │
│ rectal_temperature      │ Continuous       │ Float64                         │
│ pulse                   │ Continuous       │ Float64                         │
│ respiratory_rate        │ Continuous       │ Float64                         │
│ temperature_extremities │ OrderedFactor{4} │ CategoricalValue{Int64, UInt32} │
│ mucous_membranes        │ Multiclass{6}    │ CategoricalValue{Int64, UInt32} │
│ capillary_refill_time   │ Multiclass{3}    │ CategoricalValue{Int64, UInt32} │
│ pain                    │ OrderedFactor{5} │ CategoricalValue{Int64, UInt32} │
│ peristalsis             │ 

### Resources for Part 1

- From the MLJ manual:
   - [A preview of data type specification in
  MLJ](https://alan-turing-institute.github.io/MLJ.jl/dev/getting_started/#A-preview-of-data-type-specification-in-MLJ-1)
   - [Data containers and scientific types](https://alan-turing-institute.github.io/MLJ.jl/dev/getting_started/#Data-containers-and-scientific-types-1)
   - [Working with Categorical Data](https://alan-turing-institute.github.io/MLJ.jl/dev/working_with_categorical_data/)
- [Summary](https://juliaai.github.io/ScientificTypes.jl/dev/#Summary-of-the-default-convention) of the MLJ convention for representing scientific types
- [ScientificTypes.jl](https://juliaai.github.io/ScientificTypes.jl/dev/)
- From Data Science Tutorials:
    - [Data interpretation: Scientific Types](https://juliaai.github.io/DataScienceTutorials.jl/data/scitype/)
    - [Horse colic data](https://juliaai.github.io/DataScienceTutorials.jl/end-to-end/horse/)
- [UCI Horse Colic Data Set](http://archive.ics.uci.edu/ml/datasets/Horse+Colic)

### Exercises for Part 1

#### Exercise 1

Try to guess how each code snippet below will evaluate:

In [27]:
scitype(42)

Count

In [28]:
questions = ["who", "why", "what", "when"]
scitype(questions)

AbstractVector{Textual} (alias for AbstractArray{Textual, 1})

In [29]:
elscitype(questions)

Textual

In [30]:
t = (3.141, 42, "how")
scitype(t)

Tuple{Continuous, Count, Textual}

In [31]:
A = rand(2, 3)

2×3 Matrix{Float64}:
 0.362628  0.514262  0.443691
 0.905562  0.186287  0.216368

-

In [32]:
scitype(A)

AbstractMatrix{Continuous} (alias for AbstractArray{Continuous, 2})

In [33]:
elscitype(A)

Continuous

In [34]:
using SparseArrays
Asparse = sparse(A)

2×3 SparseArrays.SparseMatrixCSC{Float64, Int64} with 6 stored entries:
 0.362628  0.514262  0.443691
 0.905562  0.186287  0.216368

In [35]:
scitype(Asparse)

AbstractMatrix{Continuous} (alias for AbstractArray{Continuous, 2})

In [36]:
C = coerce(A, Multiclass)

2×3 CategoricalArrays.CategoricalArray{Float64,2,UInt32}:
 0.362628  0.514262  0.443691
 0.905562  0.186287  0.216368

In [37]:
scitype(C)

AbstractMatrix{Multiclass{6}} (alias for AbstractArray{Multiclass{6}, 2})

In [38]:
elscitype(C)

Multiclass{6}

In [39]:
v = [1, 2, missing, 4]
scitype(v)

AbstractVector{Union{Missing, Count}} (alias for AbstractArray{Union{Missing, Count}, 1})

In [40]:
elscitype(v)

Union{Missing, Count}

In [41]:
scitype(v[1:2])

AbstractVector{Union{Missing, Count}} (alias for AbstractArray{Union{Missing, Count}, 1})

Can you guess at the general behavior of
`scitype` with respect to tuples, abstract arrays and missing
values? The answers are
[here](https://github.com/juliaai/ScientificTypesBase.jl#2-the-scitype-and-scitype-methods)
(ignore "Property 1").

#### Exercise 2

Coerce the following vector to make MLJ recognize it as a vector of
ordered factors (with an appropriate ordering):

In [42]:
quality = ["good", "poor", "poor", "excellent", missing, "good", "excellent"]

7-element Vector{Union{Missing, String}}:
 "good"
 "poor"
 "poor"
 "excellent"
 missing
 "good"
 "excellent"

#### Exercise 3 (fixing scitypes in a table)

Fix the scitypes for the [House Prices in King
County](https://mlr3gallery.mlr-org.com/posts/2020-01-30-house-prices-in-king-county/)
dataset:

In [43]:
house_csv = urldownload("https://raw.githubusercontent.com/ablaom/"*
                        "MachineLearningInJulia2020/for-MLJ-version-0.16/"*
                        "data/house.csv");
house = DataFrames.DataFrame(house_csv)
first(house, 4)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition
,Float64,Int64,Float64,Int64,Int64,Float64,Int64,Int64,Int64
1,221900.0,3,1.0,1180,5650,1.0,0,0,3
2,538000.0,3,2.25,2570,7242,2.0,0,0,3
3,180000.0,2,1.0,770,10000,1.0,0,0,3
4,604000.0,4,3.0,1960,5000,1.0,0,0,5


(Two features in the original data set have been deemed uninformative
and dropped, namely `:id` and `:date`. The original feature
`:yr_renovated` has been replaced by the `Bool` feature `is_renovated`.)

<a id='part-2-selecting-training-and-evaluating-models'></a>

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*